In [1]:
#This script opens the citation dataset and perform calculations in patent level
#Renato Kogeyama


#Miami, December 29th, 2019
# The script runs in about 20 min with the full dataset - excellent!
# The main change was in the philosophy of the code - ditching for loops
# This version includes application data calculate time variables 
# The idea is to evaluate changes in patent policy and evaluate time to citation

#Miami, December 27th, 2019
# Due bad performance, I am updating the script to more efficient coding

# This script generates a citation ouput with patent|backward citation|cumulated backward citation | year
# The output generates a citation record by year, considering direct and indirect citation


In [2]:
import pandas as pd
import re

import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler


In [3]:
%matplotlib inline

In [4]:
fname = r'/home/rkogeyam/PATENT_CITATION/data/uspatentcitation.tsv'
dst = '/home/rkogeyam/PATENT_CITATION/data/cit_tree.csv'

In [5]:
%%time
df=pd.read_csv(fname)

# sample_size=100
# df=sampler(fname, sample_size, sep='\t')

CPU times: user 3min 44s, sys: 15.6 s, total: 4min
Wall time: 4min


In [6]:
df.head()

,uuid,patent_id,citation_id,date,name,kind,country,category,sequence
0,000007b7c0x3n9iy1othb9hz7,9009250,8127342,2012-02-01,Boynton et al.,B2,US,cited by applicant,622
1,000007obbnxmqc6v43t1c00dw,9643605,5471515,1995-11-01,Fossum et al.,A,US,cited by applicant,413
2,00000jd7thmiucpaol1hm1835,5354551,4875247,1989-10-01,Berg,NaN,US,NaN,11
3,00000oohcwply47niwhm5i42l,D786922,D718330,2014-11-01,Shin,S,US,cited by examiner,11
4,00002ghv7v87txbmryocp9i60,D490798,D190749,1961-06-01,Matri et al.,S,US,cited by examiner,1


In [7]:
df['patent_id']=df['patent_id'].astype(str)
df['citation_id']=df['citation_id'].astype(str)

In [8]:
df.dtypes

uuid           object
patent_id      object
citation_id    object
date           object
name           object
kind           object
country        object
category       object
sequence        int64
dtype: object

In [9]:
df.shape

(91453297, 9)

In [10]:
# some data cleaning

# here I should keep the information for further analysis

# drop rows in which column citation_id has a length smaller than 5
indexNames = df[df['citation_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

# drop rows for which column patent_id has a length smaller than 5
indexNames = df[df['patent_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [11]:
df.shape

(91445558, 9)

In [12]:
%%time
# # Keep this for reference!
# # As of Dec 31st, 2019, I compared the clean to the raw version of citation and patent ids

# #stripping non-desired characters but keeping the originals for later check - only three changes in citation_id

df['citation_id_raw']=df['citation_id']
df['patent_id_raw']=df['patent_id']

cleaning_patent=lambda x:re.sub('([^a-zA-Z0-9]+)', "", x)
df['citation_id']=df['citation_id'].apply(cleaning_patent)
df['patent_id']=df['patent_id'].apply(cleaning_patent)

# #stripping trailing white spaces - no changes
# df['patent_id'] = df['patent_id'].str.strip()
# df['citation_id'] = df['citation_id'].str.strip()

# #this is taking a lot of time, evaluate alternatives

print(df[df['citation_id'].map(len) > 8])

                               uuid patent_id  citation_id        date  \
1115      000l397rmcu1p9fxvegt75gd8   6834474  20020090489  2002-07-01   
1124      000l6y5a355o1xp8kw12zat2q   6854492  20010025668  2001-10-01   
3978      0021zkyqbzv8dp89k0e9i5pvn   D474084  20010013500  2001-08-01   
4915      002jwbj8ch96kavoxjhkw8czo   6738820  20020031120  2002-03-01   
4990      002ld67k6ty2it7oo6legcqh5   6862552  20020034191  2002-03-01   
5011      002lpkh2rawkxxp7gm2q3ye8u   6679612  20030002180  2003-01-01   
5183      002omicihgubu2nt4x2kpzakb   6614073  20020053696  2002-05-01   
6217      0037xjdpdapjlidqzlgh9gqop   6622391  20020153479  2002-10-01   
7691      003zfsqsbkkitu3ccrb94glu8   6809520  20020030492  2002-03-01   
10455     005dy0m3n9516z5eepmi2crm3   6822677  20010009438  2001-07-01   
10735     005izkfzlac5jfgtfbqid3ybb   6739551  20030127562  2003-07-01   
11196     005shkiewmjkql7i9jf1litjy   6868406  20010055388  2001-12-01   
11610     006150t0ybq0jq8glp95epfa8   

In [13]:
print(df[df['patent_id'].map(len) > 8])

Empty DataFrame
Columns: [uuid, patent_id, citation_id, date, name, kind, country, category, sequence, citation_id_raw, patent_id_raw]
Index: []


In [14]:
print(df[df['patent_id']!=df['patent_id']])

Empty DataFrame
Columns: [uuid, patent_id, citation_id, date, name, kind, country, category, sequence, citation_id_raw, patent_id_raw]
Index: []


In [15]:
%%time
#Non-conforming dates
#this corrects days or months registered as 00
df.date.replace({'-00':'-01'}, regex=True, inplace=True)

#replacing day or month would not affect the output
#replacing the year could be more problematic

#Ditch month and day altogether, since it is not a reliable source of information

CPU times: user 1min 12s, sys: 80 ms, total: 1min 12s
Wall time: 1min 12s


In [16]:
%%time
#attention to date format - original data is year-month-day
df['date']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce') 

CPU times: user 35.5 s, sys: 1.65 s, total: 37.1 s
Wall time: 37 s


In [17]:
%%time
#cited patents registers total citations received 

cited_patents=df.groupby(['citation_id']).count().iloc[:, 1].reset_index() #Series, patent-level
# cited_patents.dropna(0, inplace=True) #Series, '0' implies that rows are dropped

#I should check this, because later I make citation back as index for merging purposes
# cited_patents=cited_patents.reset_index() #Dataframe

CPU times: user 1min 54s, sys: 3.81 s, total: 1min 57s
Wall time: 1min 57s


In [18]:
cited_patents.rename(columns={'patent_id': 'back_citation'}, inplace=True)

In [19]:
df.head()

,uuid,patent_id,citation_id,date,name,kind,country,category,sequence,citation_id_raw,patent_id_raw
0,000007b7c0x3n9iy1othb9hz7,9009250,8127342,2012-02-01,Boynton et al.,B2,US,cited by applicant,622,8127342,9009250
1,000007obbnxmqc6v43t1c00dw,9643605,5471515,1995-11-01,Fossum et al.,A,US,cited by applicant,413,5471515,9643605
2,00000jd7thmiucpaol1hm1835,5354551,4875247,1989-10-01,Berg,NaN,US,NaN,11,4875247,5354551
3,00000oohcwply47niwhm5i42l,D786922,D718330,2014-11-01,Shin,S,US,cited by examiner,11,D718330,D786922
4,00002ghv7v87txbmryocp9i60,D490798,D190749,1961-06-01,Matri et al.,S,US,cited by examiner,1,D190749,D490798


In [20]:
#forward citation
forward_citation=df.groupby(['patent_id']).count().iloc[:, 1].reset_index()
forward_citation.rename(columns={'citation_id': 'forw_citation'}, inplace=True)



In [21]:
forward_citation.head()

,patent_id,forw_citation
0,3930271,2
1,3930272,11
2,3930273,8
3,3930274,3
4,3930275,2


In [22]:
%%time
#this is the trick code
#I match the total of citations received by the citing patent (identified by patent_id)
#Then I sum all citations received by the citing patents

df=df.merge(cited_patents, how='inner', left_on='patent_id', right_on='citation_id')
df.rename(columns={'back_citation': 'parent_back_citation'}, inplace=True)

CPU times: user 5min 13s, sys: 9.32 s, total: 5min 22s
Wall time: 5min 22s


In [23]:
%%time
#merging generates new NaNs
#NaN in citations means no citation
df.fillna(0, inplace=True)

CPU times: user 2min 40s, sys: 3.16 s, total: 2min 43s
Wall time: 2min 43s


In [24]:
%%time
#now I collapse df to become a patent level dataframe
#citation_id_x is the original cited patent
patent_level_df=df.groupby('citation_id_x').sum()

CPU times: user 1min 45s, sys: 2.65 s, total: 1min 48s
Wall time: 1min 48s


In [25]:
# now i merge the number of citations to patent_level_df
# patent_level has the information about the citing patent

outcome=cited_patents.merge(patent_level_df, how='outer', left_on='citation_id', right_index=True)

In [26]:
outcome.head()

,citation_id,back_citation,sequence,parent_back_citation
0,0000000,1,0.0,10.0
1,000004,1,0.0,1.0
2,000006,1,0.0,11.0
3,0000482,1,0.0,30.0
4,0000H94,1,0.0,14.0


In [27]:
outcome=outcome.merge(forward_citation, how='outer', left_on='citation_id', right_on='patent_id')

In [28]:
outcome.head()

,citation_id,back_citation,sequence,parent_back_citation,patent_id,forw_citation
0,0000000,1.0,0.0,10.0,NaN,NaN
1,000004,1.0,0.0,1.0,NaN,NaN
2,000006,1.0,0.0,11.0,NaN,NaN
3,0000482,1.0,0.0,30.0,NaN,NaN
4,0000H94,1.0,0.0,14.0,NaN,NaN


In [29]:
outcome.fillna(0, inplace=True)

In [30]:
outcome.head()

,citation_id,back_citation,sequence,parent_back_citation,patent_id,forw_citation
0,0000000,1.0,0.0,10.0,0,0.0
1,000004,1.0,0.0,1.0,0,0.0
2,000006,1.0,0.0,11.0,0,0.0
3,0000482,1.0,0.0,30.0,0,0.0
4,0000H94,1.0,0.0,14.0,0,0.0


In [31]:
outcome.drop(['sequence', 'patent_id'], axis=1, inplace=True)

In [ ]:
outcome.set_index('citation_id')

In [32]:
outcome.describe()

,back_citation,parent_back_citation,forw_citation
count,8.525072e+06,8.525072e+06,8.525072e+06
mean,1.072666e+01,1.814898e+02,1.072666e+01
std,2.819247e+01,1.365067e+03,3.634551e+01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,0.000000e+00,0.000000e+00
50%,3.000000e+00,1.400000e+01,4.000000e+00
75%,1.000000e+01,7.900000e+01,1.000000e+01
max,3.922000e+03,2.274790e+05,5.811000e+03


In [33]:
%%time
outcome.to_csv(dst)

CPU times: user 1min 9s, sys: 388 ms, total: 1min 10s
Wall time: 1min 10s


In [34]:
# I should introduce a test to see if the script is calculating correctly
# I should search for the information of one specfic patent and compare with the output generated by the script